In [ ]:
# Projekt zaliczeniowy
# Lukasz Zlocki
# 76103

In [ ]:
# stan laborki
# Zadanie 1
#  - todo <- w trakcie kopiowania :)
# Zadanie 2
#  - punkt a : done :)
#  - punkt b : done :)
#  - punkt c : todo <-zrobi Lukasz
# Zadanie 3
#  - todo <- Adrian
# Zadanie 4
#  - standardowe : done :)
#  - Zadanie z * : todo <-zrobi Lukasz
# Zadanie 5
#  - punkt a : todo <- zrobi Damian
#  - punkt b : todo <- zrobi Damian
#  - punkt c : todo <- zrobi Damian
#  - punkt d : todo <- zrobi Damian
# Zadanie 6
#  - punkt a : todo
#  - punkt b : todo
#  - punkt c : todo

In [ ]:
# ZADANIE 1
# Napisac klase, ktora umozliwia tworzenie wektorow BoW i TF-IDF

In [1]:
import numpy as np
import pandas as pd
from collections import Counter, defaultdict
from math import log
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lzloc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
class CustomVectorizer:
    def __init__(self, use_tfidf=False, tokenizer=None, stoplist=None, stemming=False):
        self.use_tfidf = use_tfidf
        self.tokenizer = tokenizer if tokenizer else self.default_tokenizer
        self.stoplist = stoplist if stoplist else set()
        self.stemming = stemming
        self.stemmer = PorterStemmer() if stemming else None
        self.vocabulary = {}
        self.idf = {}

    def default_tokenizer(self, text):
        return re.findall(r"\b\w+\b", text.lower())

    def fit(self, corpus):
        token_counts = Counter()
        doc_freq = defaultdict(int)

        for document in corpus:
            tokens = self._process_tokens(self.tokenizer(document))
            token_counts.update(tokens)
            unique_tokens = set(tokens)
            for token in unique_tokens:
                doc_freq[token] += 1

        self.vocabulary = {token: idx for idx, token in enumerate(sorted(token_counts.keys()))}

        if self.use_tfidf:
            n_docs = len(corpus)
            self.idf = {token: log((n_docs + 1) / (freq + 1)) + 1 for token, freq in doc_freq.items()}

    def transform(self, documents):
        matrix = []

        for document in documents:
            tokens = self._process_tokens(self.tokenizer(document))
            token_counts = Counter(tokens)

            row = np.zeros(len(self.vocabulary))
            for token, count in token_counts.items():
                if token in self.vocabulary:
                    if self.use_tfidf:
                        row[self.vocabulary[token]] = count * self.idf.get(token, 0)
                    else:
                        row[self.vocabulary[token]] = count

            matrix.append(row)

        return pd.DataFrame(matrix, columns=self.vocabulary.keys())

    def _process_tokens(self, tokens):
        processed_tokens = []
        for token in tokens:
            if token in self.stoplist:
                continue
            if self.stemming:
                token = self.stemmer.stem(token)
            processed_tokens.append(token)
        return processed_tokens

In [3]:
# Test klasy
corpus = [
        "This is a sample document.",
        "This document is another example document.",
        "Text processing with BoW and TF-IDF is fun!"
    ]
stoplist = set(stopwords.words("english"))

# BoW
bow_vectorizer = CustomVectorizer(stoplist=stoplist)
bow_vectorizer.fit(corpus)
bow_vectors = bow_vectorizer.transform(corpus)
print("BoW Vectors:")
print(bow_vectors)

# TF-IDF
tfidf_vectorizer = CustomVectorizer(use_tfidf=True, stoplist=stoplist)
tfidf_vectorizer.fit(corpus)
tfidf_vectors = tfidf_vectorizer.transform(corpus)
print("\nTF-IDF Vectors:")
print(tfidf_vectors)

BoW Vectors:
   another  bow  document  example  fun  idf  processing  sample  text   tf
0      0.0  0.0       1.0      0.0  0.0  0.0         0.0     1.0   0.0  0.0
1      1.0  0.0       2.0      1.0  0.0  0.0         0.0     0.0   0.0  0.0
2      0.0  1.0       0.0      0.0  1.0  1.0         1.0     0.0   1.0  1.0

TF-IDF Vectors:
    another       bow  document   example       fun       idf  processing  \
0  0.000000  0.000000  1.287682  0.000000  0.000000  0.000000    0.000000   
1  1.693147  0.000000  2.575364  1.693147  0.000000  0.000000    0.000000   
2  0.000000  1.693147  0.000000  0.000000  1.693147  1.693147    1.693147   

     sample      text        tf  
0  1.693147  0.000000  0.000000  
1  0.000000  0.000000  0.000000  
2  0.000000  1.693147  1.693147  


In [ ]:
# ZADANIE 2.a
# Wyszukiwarka dokumentow ze zbioru 20newsgroups
# Podobienstwo cosinusowe

In [3]:
from sklearn.datasets import fetch_20newsgroups 
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity 
import numpy as np

In [4]:
# Wczytanie danych 20newsgroups
newsgroups = fetch_20newsgroups(subset='all')
documents = newsgroups.data

documents

# Stwórz wektor TF-IDF dla dokumentów
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(documents)

In [7]:
#Podobienstwo cosinusowe
def get_similar_docs_by_cosine(query_string, top_docs_quantity):
    '''
    Retrive documents with cosine similarity
    
    parameters:
        query_string - string with query
        top_docs_quantity - quantity of top similar documents to print
    
    returns:
    list: A list of tuples, each containing a document and its cosine similarity score.
    '''
    # Przekształcenie zapytania w wektor TF-IDF 
    query_tfidf = vectorizer.transform([query_string]) 
    # Obliczenie podobieństwa cosinusowego pomiędzy zapytaniem a dokumentami 
    cosine_similarities = cosine_similarity(query_tfidf, tfidf_matrix).flatten() 
    # Znajdź indeksy dokumentów o największym podobieństwie kosinusowym 
    top_indices = cosine_similarities.argsort()[-top_docs_quantity:][::-1] 
    # Zwróć posortowaną listę dokumentów 
    return [(documents[i], cosine_similarities[i]) for i in top_indices]

In [8]:
# Przykładowe zapytanie 1 - podobienstwo cosiunusowe
query = "reading Bible" 
top_documents = get_similar_docs_by_cosine(query, 5) 
# Wypisz wyniki 
print(f"Top 5 most similar news to query: " + query)
for doc, score in top_documents: 
    print(f"------------------------------")
    print(f"Similarity: {score:.4f}")
    print(f"{doc[:500]}\n---\n")
    print(f"------------------------------")

Top 5 most similar news to query: reading Bible
------------------------------
Similarity: 0.4067
From: spebcg@thor.cf.ac.uk (BCG)
Subject: Re: Knowing God's Will
Organization: uwcc
Lines: 20

Hi,

I don't know much about Bible. Could you tell me the relations of
Christians with non-Christians in Bible? How should be The relations of
christian nations with each other and the relations of Christian nations
with other nations who are not Christians?

The other question is about the concept of religion in Bible. Does the
religion of God include and necessitate any law to be extracted from
Bible
---

------------------------------
------------------------------
Similarity: 0.3988
From: halsall@murray.fordham.edu (Paul Halsall)
Subject: Bible Unsuitable for New Christians
Reply-To: halsall@murray.fordham.edu
Organization: J. Random Misconfigured Site
Lines: 42


	A "new Christian" wrote that he was new to the faith and 
learning about it "by reading the Bible, of course". I am not
at all su

In [9]:
# Przykladowe zapytanie 2 - podobienstwo cosiunusowe
query = "toyota and honda are the best cars" 
top_documents = get_similar_docs_by_cosine(query, 5) 
# Wypisz wyniki 
print(f"Top 5 most similar news to query: " + query)
for doc, score in top_documents: 
    print(f"------------------------------")
    print(f"Similarity: {score:.4f}")
    print(f"{doc[:500]}\n---\n")
    print(f"------------------------------")

Top 5 most similar news to query: toyota and honda are the best cars
------------------------------
Similarity: 0.3241
From: carl_f_hoffman@cup.portal.com
Subject: 1993 Infiniti G20
Organization: The Portal System (TM)
Lines: 26


I am thinking about getting an Infiniti G20.
In consumer reports it is ranked high in many
catagories including highest in reliability index for compact cars.
Mitsubushi Galant was second followed by Honda Accord).

A couple of things though:
1) In looking around I have yet to see anyone driving this
   car. I see lots of Honda's and Toyota's.
2) There is a special deal where I can get
---

------------------------------
------------------------------
Similarity: 0.2480
From: gt4722a@prism.gatech.EDU (James B. Atkins)
Subject: Honda Mailing list?
Organization: Georgia Institute of Technology
Lines: 2


	Is there a Honda mailing list, and if so how do I subscribe to it?

---

------------------------------
------------------------------
Similarity: 0.2246
From

In [ ]:
# ZADANIE 2a - wnioski:

# Dzieki funkcji get_similar_docs_by_cosine obliczylem miare podobienstw miedzy zapytaniem (query) oraz kazdym dokumentem w korpusie.
# Wyniki zostaly posortowane od wartosci najwiekszej do najmniejszej po wartosci podobienstwa(similarity).
# Wyswietlone wyniki powyzej pokazuja maile o najwiekszym podobienstwie do zadanego zapytania.
# W przypadku zapytan "reading Bible" czy "toyota and honda are the best cars"  wskazane zostaly prawidlowo emaile dotyczace danego zagadnienia
# Stopien podobienstwa wyswietlonych emaili 0.2 - 0.22 oraz 0.39 - 0.42. 
# Przy tym stopniu podobienstwa 0.39 - 0.42.  prawidlowo oceniam dobor emaili do zadanego zapytania (query) 
# Zauwazylem , ze przy stopniu podobienstwa 0.2 - 0.22 ten dobor maili tez oceniam jako prawidlowy i zgodny z tematem.

In [ ]:
# ZADANIE 2.b
# Wyszukiwarka dokumentow ze zbioru 20newsgroups
# iloczyn skalarny zamiast podobienstwa kosinusowego

In [10]:
def get_similar_docs_by_scalar(query, vectorizer, tfidf_matrix, top_n): 
    '''
    Retrive similar documents by scalar similarity
    parameters:
        query (str): String containing the query to search for similar documents. 
        vectorizer: TF-IDF vectorizer used to transform the query and documents into vector form. 
        tfidf_matrix: TF-IDF matrix representing the collection of documents. 
        top_n (int): Number of top similar documents to retrieve.
    Returns: 
        top_indices (array): Indices of the top n most similar documents. 
        scalar_products (array): Scalar similarity scores of the top n documents. 
    '''  
    # Konwersja zapytania do wektora TF-IDF 
    query_vector = vectorizer.transform([query]) 
    # Obliczenie iloczynu skalarnego między zapytaniem a wszystkimi dokumentami 
    scalar_products = np.dot(query_vector, tfidf_matrix.T).toarray().flatten() 
    # Znalezienie indeksów dokumentów o najwyższym iloczynie skalarnym 
    top_indices = scalar_products.argsort()[-top_n:][::-1] 
    return top_indices, scalar_products[top_indices]

In [11]:
# Wczytanie danych 20newsgroups
newsgroups = fetch_20newsgroups(subset='all')
documents = newsgroups.data

# Stwórz wektor TF-IDF dla dokumentów
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(documents)

# Przykladowe zapytanie:
query = "toyota and honda are the best cars"

#Iloczyn skalarny podobienstw
top_indices_skalar, top_scores_skalar = get_similar_docs_by_scalar(query, vectorizer, tfidf_matrix, 5)

# Wyświetlenie wyników 
for doc_skalar, score_skalar in zip(top_indices_skalar, top_scores_skalar): 
    print(f"Similarity: {score_skalar:.4f}\n{documents[doc_skalar][:500]}\n---\n")

Similarity: 0.3241
From: carl_f_hoffman@cup.portal.com
Subject: 1993 Infiniti G20
Organization: The Portal System (TM)
Lines: 26


I am thinking about getting an Infiniti G20.
In consumer reports it is ranked high in many
catagories including highest in reliability index for compact cars.
Mitsubushi Galant was second followed by Honda Accord).

A couple of things though:
1) In looking around I have yet to see anyone driving this
   car. I see lots of Honda's and Toyota's.
2) There is a special deal where I can get
---

Similarity: 0.2480
From: gt4722a@prism.gatech.EDU (James B. Atkins)
Subject: Honda Mailing list?
Organization: Georgia Institute of Technology
Lines: 2


	Is there a Honda mailing list, and if so how do I subscribe to it?

---

Similarity: 0.2246
From: srihari@cirrus.com (Srihari Shoroff)
Subject: Re: Instead of a Saturn SC2, What???
Organization: Cirrus Logic Inc.
Distribution: na
Lines: 24

In <Ifn=sPO00iV18_A8NZ@andrew.cmu.edu> jr4q+@andrew.cmu.edu (Jason M. Roth) wri

In [ ]:
# ZADANIE 2b - wnioski:
# W punkcie b zastosowalem iloczyn skalarny podobienstwa , wynik w porownaniu do podobiensta cosinusowego i dla tego samego
# zapytania "toyota and honda are the best cars" jest identyczny. obliczony wskaznik podobienstwa tez jest na tym samym poziomie.

In [12]:
# ZADANIE 4
# Dla każdej grupy ze zbioru  20newsgroups wypisz inną grupę najbardziej i najmniej podobną znaczeniowo.

In [13]:
# Function to calculate the average cosine similarity between two groups
def get_average_cosine_similarity(group1_indices, group2_indices): 
    """
    Calculate the average cosine similarity between two groups of documents.

    Parameters: 
        group1_indices : array-like Indices of the documents in the first group. 
        group2_indices : array-like Indices of the documents in the second group. 
        
    Returns: 
        float: The average cosine similarity between the two groups of documents.   
    """
    # Use cosine_similarity from sklearn
    similarities = cosine_similarity(tfidf_matrix[group1_indices], tfidf_matrix[group2_indices]) 
    return np.mean(similarities)

In [14]:
# Load needed libraries
from sklearn.datasets import fetch_20newsgroups 
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity 
import numpy as np

In [15]:
# Read data from 20newsgroups
newsgroups = fetch_20newsgroups(subset='all')

# Divide documents between categories and labels
documents = newsgroups.data
categories = newsgroups.target_names 
labels = newsgroups.target

# Transform text to TF-IDF vectors
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(documents)

#Print categories
print("Categories:") 
categories


Categories:


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [16]:
# Calculating average cosine similarity for all pairs of groups
group_similarities = {} 
for i in range(len(categories)): 
    for j in range(i+1, len(categories)): 
        group1_indices = np.where(labels == i)[0] 
        group2_indices = np.where(labels == j)[0] 
        similarity = get_average_cosine_similarity(group1_indices, group2_indices) 
        group_similarities[(categories[i], categories[j])] = similarity

# Indicate most and least similar groups
most_similar_groups = max(group_similarities, key=group_similarities.get) 
least_similar_groups = min(group_similarities, key=group_similarities.get)

# Printing results
print("MOST similar groups:")
print(f"{most_similar_groups}")
print(f"similarity:") 
print(f"{group_similarities[most_similar_groups]}")
print("")
print("LEAST similar groups:")
print(f"{least_similar_groups}")
print("similarity:")
print(f"{group_similarities[least_similar_groups]}")

MOST similar groups:
('soc.religion.christian', 'talk.religion.misc')
similarity:
0.02346466873647374

LEAST similar groups:
('comp.windows.x', 'rec.sport.hockey')
similarity:
0.0071516829474918995


In [ ]:
# ZADANIE 4 - wnioski:
# Przy uzyciu metody kosinusowego prawdopodobienstwa wyznaczono grupy najbardziej oraz najmniej podobne
# Grupy najbardziej podobne (similarity: 0.02346) to grupy 'soc.religion.christian', 'talk.religion.misc' i tu widac , ze grupy zwiazane sa z religia
# Grupy najmniej podobne (similarity: 0.0071) to grupy 'comp.windows.x', 'rec.sport.hockey' hokej oraz computery/windows to sa rozne tematy.
# Zadanie zakonczone sukcesem